In [2]:
#import math
#import itertools
import numpy as np
import matplotlib.pyplot as plt

#import plotly
#import plotly.express as px

from mpl_toolkits import mplot3d
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
import tqdm
#from functools import lru_cache
#import re
import networkx as nx
#from networkx.algorithms import approximation


import MDAnalysis as mda

#import scipy
#import sklearn
#import skimage

#import xml.etree.ElementTree as et
#from Bio.PDB import *
#import nglview as nv

from timeit import default_timer as timer


import torch



# http://nglviewer.org/nglview/latest/api.html
# https://biopython.org/wiki/The_Biopython_Structural_Bioinformatics_FAQ
# https://ambermd.org/tutorials/analysis/tutorial_notebooks/nglview_notebook/index.html
# https://amber-md.github.io/pytraj/latest/_api/pytraj.html

In [3]:
#contact_matrix = np.loadtxt('/home/lillo/TesiCNTE/CNTE/dataset/contact_matrix.txt')   #laptop
#contact_matrix = np.loadtxt('/home/lillo/Code/Tesi/dataset/contact_matrix.txt')        #fisso
#contact_matrix_single = contact_matrix.reshape(100,100,12,12)

#gromacs_output = open('/home/lillo/Code/Tesi/dataset/dm4500Compl_mix1_K2_1%4500ns.gro') #fisso
#gromacs_output = open('/home/lillo/TesiCNTE/CNTE/dataset/dm4500Compl_mix1_K2_1%4500ns.gro') #laptop

#path = '/home/lillo/Code/Tesi/dataset/dm4500Compl_mix1_K2_1%4500ns.gro' #fisso
path = '/home/lillo/TesiCNTE/CNTE/dataset/dm4500Compl_mix1_K2_1%4500ns.gro' #laptop

# import 2mxu file (beta sheet)

#path_to_mmCIF = open('/home/lillo/TesiCNTE/pdb/2mxu/2mxu.cif')  ## laptop
#path_to_pdb = '/home/lillo/TesiCNTE/pdb/2mxu/2mxu.pdb'  ## laptop
#pa_to_pdb = '/home/lillo/TesiCNTE/pdb/2mxu/2mxu.pdb'  ## laptop

#path_to_mmCIF = open('/home/lillo/Code/Tesi/pdb/2mxu/2mxu.cif')  ## fisso
#path_to_pdb = '/home/lillo/Code/Tesi/pdb/2mxu/2mxu.pdb'  ## fisso
#pa_to_pdb = '/home/lillo/Code/Tesi/pdb/2mxu/2mxu.pdb'  ## fisso

# aggregate blob

seed_1_path = '/home/lillo/TesiCNTE/from_cluster/aggregate1.gro' # laptop
#seed_1_path = '/home/lillo/Code/Tesi/dataset/aggregate1.gro' # Fisso

# Trajectory with aggregate seed
trj_xtc = '/home/lillo/TesiCNTE/CNTE/trajectory/prd-LDLK12-100mer-out-mol.xtc'  #laptop
trj_gro = '/home/lillo/TesiCNTE/CNTE/trajectory/min-LDLK12-100mer-out-c.gro'  #laptop

#trj_xtc = '/home/lillo/Code/Tesi/dataset/trajectory_6_12_19/prd-LDLK12-100mer-out-mol.xtc'  #fisso
#trj_gro = '/home/lillo/Code/Tesi/dataset/trajectory_6_12_19/min-LDLK12-100mer-out-c.gro'  #fisso


In [ ]:
# READ .gro FILE AND PREPROCESSING

def clean_gro(path):
    
    
        # open file .gro and return a list with one element per line of the .gro file
    def read_gro(path):
        gromacs_output = open(path)

        gro_file = []
        for line in tqdm.tqdm(gromacs_output):
            gro_file.append(line)



        gromacs_output.close()        

        return gro_file



    # return string in a string with numbers
    def return_if_string(string):
        digits = []
        for i in string:
            if not i.isdigit():
                digits.append(i)

        string = ''.join(digits)

        return string


    # return numbers in a string with numbers
    def return_if_digit(string):
        digits = []
        for i in string:
            if i.isdigit():
                digits.append(i)

        string = ''.join(digits)

        return string


    # remove first, second and last lines from gro_file and reorder information
    # FIX OPTION TO GET ENTRY RELATED TO A LABEL (as 'bb' or 'ca')
    def clean_gro_file(gro_file):
        cleaned_gro_file = []
        for aminoacid in tqdm.tqdm(gro_file[2:-1]):
            splitted = aminoacid.split()
            if splitted[1] == 'BB':
                position_in_peptide = return_if_digit(splitted[0])
                residue = return_if_string(splitted[0])
                index = splitted[2]
                x = splitted[3]
                y = splitted[4]
                z = splitted[5]
                cleaned_gro_file.append([index, position_in_peptide, residue, x, y, z])
        return cleaned_gro_file
    
    
    gro_file = read_gro(path)
    cleaned_gro_file = clean_gro_file(gro_file)

    return cleaned_gro_file


# create coordinate dict from cleaned_gro_file
def get_coordinate_dict_from_cleaned_gro(cleaned_gro_file):
    
    peptide_lenght_list = []

    temporary_list = []

    # iterate trough cleaned_gro_file
    for residue in cleaned_gro_file:

        # if temporary list just started, add aminoacid position in chain
        if len(temporary_list) == 0:
            temporary_list.append(int(residue[1]))

        else:
            # if position of actual residue is less than last residue
            if temporary_list[-1] > int(residue[1]):

                # append lenght of last peptide to peptide lenght list
                peptide_lenght_list.append(len(temporary_list))

                # empty temporary list
                temporary_list = []

                # append actual residue position
                temporary_list.append(int(residue[1]))

            # if position of actual residue is higher than last residue, ad current residue position
            else:
                temporary_list.append(int(residue[1]))

    # append last peptide lenght to lenght stack
    peptide_lenght_list.append(len(temporary_list))

    # create empty dict for coordinate
    peptide_coordinate_dict = {}

    # create an entry in dict for every peptide in the file
    for peptide in range(len(peptide_lenght_list)):
        peptide_coordinate_dict[peptide] = {}

        # for every residue in lenght peptide, add coordinate x, y, z
        for residue in range(peptide_lenght_list[peptide]):
            peptide_coordinate_dict[peptide][residue] = [float(coordinate) for coordinate in cleaned_gro_file[(peptide * peptide_lenght_list[peptide])+residue][3:]]

    return peptide_coordinate_dict


# compute euclidean distance
def get_euclidean_distance(point_1, point_2):
    
    euclidean_distance = np.sqrt(np.sum([((point_1[0] - point_2[0])**2), ((point_1[1] - point_2[1])**2), ((point_1[2] - point_2[2])**2)]))

    return euclidean_distance

# compute distance map between two peptides
def compute_distance_map(coordinate_dict, peptide_1, peptide_2):

    distance_map = []
    for amino_1 in coordinate_dict[peptide_1]:
        coordinate_1 = coordinate_dict[peptide_1][amino_1]
        
        distance_map.append([amino_1])
        
        for amino_2 in coordinate_dict[peptide_2]:
            coordinate_2 = coordinate_dict[peptide_2][amino_2]
            
            euclidean_distance = get_euclidean_distance(coordinate_1, coordinate_2)
            distance_map[amino_1].append(euclidean_distance)
        
        del distance_map[amino_1][0]

    distance_map = np.asarray(distance_map)
    
    return distance_map

# compute distance map and return a n_peptide x n_peptide x n_res x n_res array
def compute_distance_maps_from_coordinate_dict(coordinate_dict):
    
    aggregate_distance_map = []

    #for peptide_1 in tqdm.tqdm(coordinate_dict):
    for peptide_1 in coordinate_dict:
        aggregate_distance_map.append([peptide_1])
        
        #for peptide_2 in tqdm.tqdm(coordinate_dict):
        for peptide_2 in coordinate_dict:
            distance_map = compute_distance_map(coordinate_dict, peptide_1, peptide_2)
            
            aggregate_distance_map[peptide_1].append(distance_map)

        del aggregate_distance_map[peptide_1][0]

    aggregate_distance_array = np.asarray(aggregate_distance_map)
    
    return aggregate_distance_array


# COMPUTE CONTACT MAPS
# TO DO: parametrize the threshold distance in a better way (e.g. )
def compute_contact_maps_as_array(distance_maps_array):
    
    # distance between the first and the second aminoacid of the first chain
    intrapeptide_minimum_distance = distance_maps_array[0][0][0][1] 

    contact_map_list = []

    # contact is in a distance up to 150% of the intrapeptide_minimum_distance [TO IMPROVE!!!]
    threshold_distance = (intrapeptide_minimum_distance * 1.5)

    for model_1 in range(distance_maps_array.shape[0]):
        contact_map_list.append([])
        for model_2 in range(distance_maps_array[model_1].shape[0]):

            contact_map_list[model_1].append([])

            if model_1 == model_2:

                contact_map_list[model_1][model_2].extend(np.zeros((distance_maps_array.shape[2], distance_maps_array.shape[3])))

            else:

                contact_map = np.zeros((distance_maps_array.shape[2], distance_maps_array.shape[3]))

                for chain_1 in range(distance_maps_array[model_1][model_2].shape[0]):

                    for chain_2 in range(distance_maps_array[model_1][model_2][chain_1].shape[0]):

                        distance = distance_maps_array[model_1][model_2][chain_1][chain_2]

                        if distance < threshold_distance:
                            contact_map[chain_1][chain_2] = 1 #True
                        else:
                            pass

                contact_map_list[model_1][model_2].extend(contact_map)
    
    contact_array = np.asarray(contact_map_list)
            
    return contact_array


#### ANALYSIS

def shift_library_maker(contact_map_to_analyze):
    
    ''' riceve numero di righe e di colonne
    restituisce un array shape((((row + col)*2)-2),row,col).
    ogni slice è una diagonale. Lo stack copre le diagonali su tutta la matrice'''
    
    row = contact_map_to_analyze.shape[0]
    col = contact_map_to_analyze.shape[1]
    
    kron_dict = {}
    kron_list_parallel = []
    kron_list_antiparallel = []
    
    for e in range(-row+1, col):
        array = np.eye(row, col, e)
        kron_list_parallel.append(array)
        kron_list_antiparallel.append(np.fliplr(array))
        
    kron_array_parallel = np.asarray(kron_list_parallel)
    kron_array_antiparallel = np.asarray(kron_list_antiparallel)
    
    kron_dict['parallel'] = kron_array_parallel
    kron_dict['antiparallel'] = kron_array_antiparallel
    
    return kron_dict


def normalized_cross_correlation_function(contact_map):
    '''
    Calculate normalized cross correlation function between a contact map and an ideal map.
    
    Arguments : contact map, as output from get_contact_maps function
                shift_matrix_stack, as output from shift_matrix_maker function
                
    Return : a list [ncc_value, index (in the shift_matrix_stack) of the shift matrix
                that is matching the contact map
            
            '''
    shift_matrix_library = shift_library_maker(contact_map)
    
    cross_correlation_values = []
    max_val = []
    sum_contact_map = np.sum(contact_map)
    
    if sum_contact_map < 2:
        pass
    
    else:
        for sense in shift_matrix_library:
            for index, z in enumerate(shift_matrix_library[sense]):

                shift_matrix = shift_matrix_library[sense][index]
                sum_shift_matrix = np.sum(shift_matrix)
                ncc_value = (np.sum((contact_map * shift_matrix))/((np.sqrt(sum_contact_map))*(np.sqrt(sum_shift_matrix))))  # normalized cross correlation function of contact matrix and shift matrix
                cross_correlation_values.append([ncc_value, index, sum_contact_map, sense])

            max_val = max(cross_correlation_values) # get only the best match (highest value of ncc)

    return max_val



def normalized_cross_correlation_for_dataset(contact_array):
    '''Calculate normalized cross correlation function between the full contacts map and and the .
    
    Arguments : contact map, as output from get_contact_maps function
                shift_matrix_stack, as output from shift_matrix_maker function
                
    Return : a list [ncc_value, index (in the shift_matrix_stack) of the shift matrix that is matching the contact map'''
    
    contact_dict = {}
    
    #for row in tqdm.tqdm(range(contact_array.shape[0])):
    for row in range(contact_array.shape[0]):
    
        for col in range((row+1), contact_array.shape[1]):
        #for col in range(contact_array.shape[1]):

            best_match = []
            best_match = normalized_cross_correlation_function(contact_array[row][col])
            
            if len(best_match) == 0:
                pass
            
            else:
                if row in contact_dict:
                    contact_dict[row].append([row, col, best_match])
                
                else:
                    contact_dict[row] = [[row, col, best_match]]
    
    return contact_dict


#denoise dataset
def denoise_full_dataset(contact_maps, normalized_cross_correlation_results):
    
    '''Denoise the contact_maps dataset using the shift_matrix
    
    Arguments : contact_maps, normalized_cross_correlation_result
    
    return : a dict with key:value = row : row, col, denoised_map
    
    '''

    denoised_dict = {}

    for peptide_1 in normalized_cross_correlation_results:
        denoised_dict[peptide_1] = {}
        for index, peptide_2 in enumerate(normalized_cross_correlation_results[peptide_1]):

            row = peptide_2[0]
            col = peptide_2[1]



            contact_map = contact_maps[row][col]
            sense = peptide_2[2][3]
            shift_matrix_index = normalized_cross_correlation_results[peptide_1][index][2][1]

            shift_matrix = shift_library_maker(contact_map)
            shift_matrix = shift_matrix[sense][shift_matrix_index]
            denoised_map = contact_map * shift_matrix

            denoised_dict[row][col] = denoised_map
            
    return denoised_dict


#create a dict that contains the peptide couples contact and the specular peptide couples contact
def reconstruct_full_matrix(denoised_dict):
    full_denoised_dict = {}
    for peptide_1 in tqdm.tqdm(denoised_dict):
        for peptide_2 in denoised_dict[peptide_1]:
            contact_map = denoised_dict[peptide_1][peptide_2]

            if peptide_1 in full_denoised_dict:
                full_denoised_dict[peptide_1][peptide_2] = contact_map

            if peptide_1 not in full_denoised_dict:
                full_denoised_dict[peptide_1] = {peptide_2:contact_map}

            if peptide_2 in full_denoised_dict:
                full_denoised_dict[peptide_2][peptide_1] = contact_map.T

            if peptide_2 not in full_denoised_dict:
                full_denoised_dict[peptide_2] = {peptide_1:contact_map.T}
    
    return full_denoised_dict


# take array, return vector with sum along columns
def get_row_vector(array):
    row_vector = np.sum(array, axis=0)
    return row_vector

# take array, return vector with sum along row
def get_col_vector(array):
    col_vector = np.sum(array, axis=1)
    return col_vector

# graph clustering
def nx_graph_search(denoised_dict):
    
    graph = nx.MultiGraph()
    
    for peptide_1 in denoised_dict:
        for peptide_2 in denoised_dict[peptide_1]:
            array_1 = denoised_dict[peptide_1][peptide_2]
            for peptide_3 in denoised_dict[peptide_2]:
                if peptide_3 != peptide_1:
                    array_2 = denoised_dict[peptide_2][peptide_3]

                    vect_1 = get_row_vector(array_1)
                    vect_2 = get_col_vector(array_2)

                    contacts = np.dot(vect_1, vect_2)
                    
                    if contacts >= 3:

                        graph.add_edge(peptide_1, peptide_2)
                     
                        graph.add_edge(peptide_2, peptide_3)

    return graph

#A novel graph clustering algorithm based on discrete-time quantum random walk
#S.G. Roya, A. Chakrabarti


# working with networkX
# if contacts >= target

# when you add_edge, nodes are created if they are not there
# you can put info in edge (as distance, n of contacts, contact map)
# you HAVE TO (but you can not also) put key to index multiple nodes that are joined with a single node
# 
# add edge from pep1 to pep2 (you HAVE TO (###to explore utility of key) put key to index multiple nodes that are joined with a single node)
# add edge from pep3 to pep3 ( same as before with key)


#FIND SUBGRAPH
def find_subgraph(graph):
    '''
    Find subgraph that have no node in common.
    
    Argument: NetworkX MultiGraph
    
    Return: list of subgraph ordered from one end to the other
    
    '''

    subgraph_list = []
    
    for node in graph:
        
        # don't explore node that are already in subgraph_list
        if node not in set(nod for nod_list in subgraph_list for nod in nod_list):
            
            # tree is the list of nodes joined to node, starting from node
            # using depht first search
            tree = [e for e in nx.algorithms.traversal.depth_first_search.dfs_tree(graph, node)]
            
            # check if the first node of the tree has adjiacency == 1
            # so it checks if it is the first or last node of the subgraph
            if len(graph[tree[0]]) == 1:
                
                if len(subgraph_list) == 0:
                    subgraph_list.append(tree)
                    
                else:
                    # use generator to check if the tree is already in the subgraph
                    if set(tree) not in (set(i) for i in subgraph_list):
                        subgraph_list.append(tree)
                        
    return subgraph_list


########## PLOT PEPTIDE LIST
# plot a list of peptide point cloud in 3d space.
# The box axis have arbitrary scale dependent on the aminoacids distance
# you can select to show the centroid
def plot_peptide_list(coordinate_dict, peptide_list, centroid=False):
    x = []
    y = []
    z = []
    x_median = float
    y_median = float
    z_median = float
    

    for peptide in range(len(peptide_list)):
        x.append([peptide])
        y.append([peptide])
        z.append([peptide])
        for aminoacid in coordinate_dict[peptide_list[peptide]]:

            point = coordinate_dict[peptide_list[peptide]][aminoacid]
            x[peptide].append(point[0])
            y[peptide].append(point[1])
            z[peptide].append(point[2])

        del x[peptide][0]
        del y[peptide][0]
        del z[peptide][0]
        
    if centroid == True:
        
        def assemble_coordinate(axis_coordinate_list):
            median_list = []
            for coordinate_set in axis_coordinate_list:
                median = np.median(coordinate_set)
                median_list.append(median)
            return median_list
        
        x_median = assemble_coordinate(x)
        y_median = assemble_coordinate(y)
        z_median = assemble_coordinate(z)
        

    #%matplotlib notebook

    fig = plt.figure()
    ax = plt.axes(projection='3d')
    
    for pep in range(len(x)):
       
        ax.scatter3D(x[pep],y[pep],z[pep])
        
        if centroid == True:
            
            ax.scatter3D(x_median[pep], y_median[pep], z_median[pep], c='red')
            
        
    #return  plt.show(), [x,y,z], [x_median, y_median, z_median]
    return  plt.show()



# get average distance map from distance maps set
def get_mean_distance_map(distance_maps):
    '''
    Calculate mean distance map from distance maps set
    
    Argument: distance maps set
    
    return: np.array with average intrapeptide distance
    
    '''
    
    # create array of zeros of shape number_of_residues * number_of_residues
    # depending on peptide residue number ### TO FIX FOR MULTIMONOMERIC ASSEMBLY
    base = np.zeros((distance_maps[0][0].shape[0], distance_maps[0][0].shape[1]))
    
    # initialize counter
    counter = 0
    
    # iterate throught peptides in the aggregate
    for peptide_1 in range(distance_maps.shape[0]):
        for peptide_2 in range(distance_maps.shape[1]):
             
            # if peptide index are the same (intrapeptide distance map)
            if peptide_1 == peptide_2:
                
                # intrapeptide distance map
                actual_distance_map = distance_maps[peptide_1][peptide_2]
                
                # sum base and current distance map
                base = base + actual_distance_map
                
                #update counter
                counter += 1

    #for element in base (every element is the sum of distance_map(i,j) for every distance map)
    for row in range(len(base)):
        for col in range(len(base)):
            
            # find the mean for every element of the cumulative distance map
            base[row][col] = (base[row][col])/counter
            
    return base


def decompose_distance_map(distance_map):
    '''Use Singular value decomposition to get
    
    distance_map.shape[1] dimensional coordinate
    (same n of dimension as the peptide n of residue)
    
    As described in:
    Mathematical Modeling of Protein Structure Using Distance Geometry
    Jeong-Mi Yoon, Yash Gad, Zhijun Wu
    
    Argument: distance map (numpy.array 2D)
    return: X : actual decomposition
            
    
    '''
    
    # initialize a zeros matrix of same shape as the input map
    D = np.zeros(distance_map.shape)
    
    #iterate trought row
    for i in range(distance_map.shape[0]):
        
        # iterate trought cols
        for j in range(distance_map.shape[1]):
            
            # distance between point point i and point j 
            dij = distance_map[i][j]
            
            # distance between point 0 and point j
            d0j = distance_map[0][j]
            
            #distance between point i and point 0
            di0 = distance_map[i][0]

            #fill the zeros matrix with the value obtained with this formula
            D[i][j] = (d0j**2 + di0**2 - dij**2)/2
            
    # check rank of matrix (should be of rank 3, but it is of rank distance_map.shape[1])
    rank = np.linalg.matrix_rank(D)
    
    # Singular value decomposition on the D matrix
    #svd = np.linalg.svd(D)
    
    svd = np.linalg.svd(D, full_matrices=False)
    
    # Calculate distance_map.shape[1] dimensional coordinate, but you need 3
    # the non necessary dimension can give data to better reconstruct the peptide structure
    X = svd[0]*np.sqrt(svd[1])

    
    #return X, svd, D, rank
    return X

def get_coordinate_from_decomposition(decomposition):
    '''Take decomposition result and convert it into a coordinate vectors dict
    
    Argument: decomposition results
    
    return: dict with reconstructed 3d coordinate vector
    
    '''
    
    # take only the first three value to compose a 3D coordinate vector
    coordinate = [e[:3] for e in decomposition]
    
    # initialize empty dict
    reconstructed_coordinate_dict = {}
    
    # fill the dict with the ccordinate vectors
    for index,coordinate_vector in enumerate(coordinate):
        reconstructed_coordinate_dict[index] = coordinate_vector
    
    return reconstructed_coordinate_dict


# 
def get_coordinate_from_distance_map(distance_map):
    ''' compute 3d coordinate from distance map
    
    Argument: distance_map (numpy.array)
    
    return: dict with 3d coordinate for every alpha-carbon of a peptide
    
    '''
    # perform singular value decomposition on distance_map (preprocessed)
    decomposed_mean_distance_map = decompose_distance_map(distance_map)
    
    
    # get 3D coordinate
    reconstructed_coordinate_dict = get_coordinate_from_decomposition(decomposed_mean_distance_map)
    
    return reconstructed_coordinate_dict

    
    
def plot_single_peptide(peptide_coordinate_dict, centroid=False):
    x = []
    y = []
    z = []
    
    for residue in peptide_coordinate_dict:
        point = peptide_coordinate_dict[residue]
        x.append(point[0])
        y.append(point[1])
        z.append(point[2])


    x = np.asarray(x)
    y = np.asarray(y)
    z = np.asarray(z)
    
    fig = plt.figure()
    ax = plt.axes(projection='3d')
    ax.scatter3D(x,y,z, c='b')
    
    if centroid == True:
            median_centroid = [np.median(x), np.median(y), np.median(z)]
            ax.scatter3D(median_centroid[0], median_centroid[1], median_centroid[2], c='r')
            
    return plt.show()


In [ ]:
# open and clean .gro file
# return list with an entry for every residue plus info
start = timer()
cleaned_gro_seed_1 = clean_gro(seed_1_path)
end = timer()
print(end - start)

# return a dict with residue coordinate
start = timer()
coordinate_dict_seed_1 = get_coordinate_dict_from_cleaned_gro(cleaned_gro_seed_1)
end = timer()
print(end - start)

# compute distance maps and return a n_peptide x n_peptide x n_res x n_res array ## from .gro coordinate dict
start = timer()
distance_maps_seed_1 = compute_distance_maps_from_coordinate_dict(coordinate_dict_seed_1)
end = timer()
print(end-start)

# compute contact maps from distance maps
start = timer()
contact_maps_seed_1 = compute_contact_maps_as_array(distance_maps_seed_1)
end = timer()
print(end-start)

# compute cross correlation and get values
start = timer()
normalized_cross_correlation_results_seed_1 = normalized_cross_correlation_for_dataset(contact_maps_seed_1)
end = timer()
print(end-start)

# denoise dataset
start = timer()
denoised_dict_seed_1 = denoise_full_dataset(contact_maps_seed_1, normalized_cross_correlation_results_seed_1)
end = timer()
print(end-start)

# recreate full contact map dataset mirrroring the upper triangle
# return dict
start = timer()
full_denoised_dict_seed_1 = reconstruct_full_matrix(denoised_dict_seed_1)
end = timer()
print(end-start)

#create graph
start = timer()
graph_seed_1 = nx_graph_search(full_denoised_dict_seed_1)
end = timer()
print(end-start)

#search subgraph
start = timer()
subgrap_list_seed_1 = find_subgraph(graph_seed_1)
end = timer()
print(end-start)

#find mean distance map
start = timer()
mean_distance_map_seed_1 = get_mean_distance_map(distance_maps_seed_1)
end = timer()
print(end-start)

# decompose mean distance matrix
start = timer()
decomposed_mean_distance_map = decompose_distance_map(mean_distance_map_seed_1)
end = timer()
print(end-start)
      
# get coordinate from distance map      
start = timer()
reconstructed_coordinate_dict = get_coordinate_from_decomposition(decomposed_mean_distance_map)
end = timer()
print(end-start)


In [ ]:
# RUN THIS FOR 3D VIEW OF PLOT

%matplotlib notebook

In [ ]:
# RUN THIS TO CLOSE ALL PLOT (run multiple time this cell if it does not work)
plt.close()

In [ ]:
# plot the reconstructed average peptide 
plot_single_peptide(reconstructed_coordinate_dict)

In [ ]:
# plot full coordinate from dict
plot_peptide_list(coordinate_dict_seed_1, [e for e in coordinate_dict_seed_1])

In [ ]:
### TRAJECTORY

# using mdanalysis module


#create Universe from a .gro with coordinates and an .xtc with the trajectory data
u = mda.Universe(trj_gro,trj_xtc)

In [ ]:
# create a dict from a Universe in which each entry is a timestep of the MD simulation
def create_trajectory_dict(u):
    bb = u.select_atoms('name BB')
    trajectory_dict = {}
    for index, time_steps in enumerate(u.trajectory):
        trajectory_dict[index] = bb.positions
    return trajectory_dict

# make trajectory dict
trajectory_dict = create_trajectory_dict(u)

In [ ]:
#if you change the number in the first parentesis, you change timestep
#second parentesis are the bb grains
pep = trajectory_dict[0][:1200] # this are all the coordinate of bb grains of timestep 0, as an array

#cast coordinate from array to dict
zero_c = get_coordinate_from_decomposition(pep)

#plot dict as a single object
plot_single_peptide(zero_c)

In [ ]:
# To see the trajectory you need to install ngl-view
# https://github.com/arose/nglview
#
# pay attention to this:
#   pip install nglview
#
# in a terminal, in the env where ngl-view is installed, send this:
# jupyter-nbextension enable nglview --py --sys-prefix
#


view = nv.show_mdanalysis(u)

In [ ]:
view

In [23]:
class morphoscanner():
    
    
    
    class math_utility():

        # take array, return vector with sum along columns
        def get_row_vector(array):
            row_vector = np.sum(array, axis=0)
            return row_vector

        # take array, return vector with sum along row
        def get_col_vector(array):
            col_vector = np.sum(array, axis=1)
            return col_vector
        
        
    class utility():
        
        def get_coordinate_dict_from_array(array):
            '''Take decomposition result and convert it into a coordinate vectors dict

            Argument: decomposition results

            return: dict with reconstructed 3d coordinate vector

            '''

            # make list with n == array.shape[0]
            coordinate = [e for e in array]

            # initialize empty dict
            reconstructed_coordinate_dict = {}

            # fill the dict with the ccordinate vectors
            for index,coordinate_vector in enumerate(coordinate):
                reconstructed_coordinate_dict[index] = coordinate_vector

            return reconstructed_coordinate_dict

    
    
    
    class read():
        
        # READ .gro FILE AND PREPROCESSING

        def clean_gro(path):


                # open file .gro and return a list with one element per line of the .gro file
            def read_gro(path):
                gromacs_output = open(path)

                gro_file = []
                for line in tqdm.tqdm(gromacs_output):
                    gro_file.append(line)



                gromacs_output.close()        

                return gro_file



            # return string in a string with numbers
            def return_if_string(string):
                digits = []
                for i in string:
                    if not i.isdigit():
                        digits.append(i)

                string = ''.join(digits)

                return string


            # return numbers in a string with numbers
            def return_if_digit(string):
                digits = []
                for i in string:
                    if i.isdigit():
                        digits.append(i)

                string = ''.join(digits)

                return string


            # remove first, second and last lines from gro_file and reorder information
            # FIX OPTION TO GET ENTRY RELATED TO A LABEL (as 'bb' or 'ca')
            def clean_gro_file(gro_file):
                cleaned_gro_file = []
                for aminoacid in tqdm.tqdm(gro_file[2:-1]):
                    splitted = aminoacid.split()
                    if splitted[1] == 'BB':
                        position_in_peptide = return_if_digit(splitted[0])
                        residue = return_if_string(splitted[0])
                        index = splitted[2]
                        x = splitted[3]
                        y = splitted[4]
                        z = splitted[5]
                        cleaned_gro_file.append([index, position_in_peptide, residue, x, y, z])
                return cleaned_gro_file


            gro_file = read_gro(path)
            cleaned_gro_file = clean_gro_file(gro_file)

            return cleaned_gro_file
        
        # create coordinate dict from cleaned_gro_file
        def get_coordinate_dict_from_cleaned_gro(cleaned_gro_file):

            peptide_lenght_list = []

            temporary_list = []

            # iterate trough cleaned_gro_file
            for residue in cleaned_gro_file:

                # if temporary list just started, add aminoacid position in chain
                if len(temporary_list) == 0:
                    temporary_list.append(int(residue[1]))

                else:
                    # if position of actual residue is less than last residue
                    if temporary_list[-1] > int(residue[1]):

                        # append lenght of last peptide to peptide lenght list
                        peptide_lenght_list.append(len(temporary_list))

                        # empty temporary list
                        temporary_list = []

                        # append actual residue position
                        temporary_list.append(int(residue[1]))

                    # if position of actual residue is higher than last residue, ad current residue position
                    else:
                        temporary_list.append(int(residue[1]))

            # append last peptide lenght to lenght stack
            peptide_lenght_list.append(len(temporary_list))

            # create empty dict for coordinate
            peptide_coordinate_dict = {}

            # create an entry in dict for every peptide in the file
            for peptide in range(len(peptide_lenght_list)):
                peptide_coordinate_dict[peptide] = {}

                # for every residue in lenght peptide, add coordinate x, y, z
                for residue in range(peptide_lenght_list[peptide]):
                    peptide_coordinate_dict[peptide][residue] = [float(coordinate) for coordinate in cleaned_gro_file[(peptide * peptide_lenght_list[peptide])+residue][3:]]

            return peptide_coordinate_dict #, peptide_lenght_list
        
        
        def get_coordinate_dict_from_cleaned_gro_for_fixed_lenght_peptides(cleaned_gro_file, peptide_lenght):
            '''Works only with system made of peptide of equal peptide length
                (the peptide in the trajectory have all the same number of aminoacids).

            Usefull to disassemble premade aggregate in a system of omogenous peptide lenght'''

            peptide_coordinate_dict = {}

            for peptide in range(len(cleaned_gro_file)//peptide_lenght):

                peptide_coordinate_dict[peptide] = {}

                peptide_data = cleaned_gro_file[(peptide*peptide_lenght):((peptide*peptide_lenght)+peptide_lenght)]

                for index, amino in enumerate(peptide_data):

                    peptide_coordinate_dict[peptide][index] = [float(amino[3]), float(amino[4]), float(amino[5])] ## can you generalize this 3? Maybe with re or isdigit function??

            return peptide_coordinate_dict
        
        
        

        
        
        
    
    
    
    class topology():
    
        
        #get a list of the number of residues of every peptide in the topology
        def get_peptide_lenght_list(topology):
            
            topology = morphoscanner.read.clean_gro(topology)
    
            peptide_lenght_list = []

            temporary_list = []

            # iterate trough topology
            for residue in topology:

                # if temporary list just started, add aminoacid position in chain
                if len(temporary_list) == 0:
                    temporary_list.append(int(residue[1]))

                else:
                    # if position of actual residue is less than last residue
                    if temporary_list[-1] > int(residue[1]):

                        # append lenght of last peptide to peptide lenght list
                        peptide_lenght_list.append(len(temporary_list))

                        # empty temporary list
                        temporary_list = []

                        # append actual residue position
                        temporary_list.append(int(residue[1]))

                    # if position of actual residue is higher than last residue, ad current residue position
                    else:
                        temporary_list.append(int(residue[1]))

            # append last peptide lenght to lenght stack
            peptide_lenght_list.append(len(temporary_list))

            return peptide_lenght_list
    
    
    
    
        def make_universe(trj_gro, trj_xtc):
            ''' Leverage MDAnalysis.Universe() to parse trajectory file from gromacs output.

            Intput: string: system path of gro file (topology) and
                            system path of xtc file (trajectory)
                            of the file to analyze

            return: MDAnalysis.Universe()'''


            universe = mda.Universe(trj_gro,trj_xtc)

            return universe
        
        
        
    
    
    
    
    
    
    
    class distance():
        
        
        # compute euclidean distance
        def get_euclidean_distance(point_1, point_2):

            euclidean_distance = np.sqrt(np.sum([((point_1[0] - point_2[0])**2), ((point_1[1] - point_2[1])**2), ((point_1[2] - point_2[2])**2)]))

            return euclidean_distance
        
        

        # compute distance map between two peptides
        def compute_distance_map(coordinate_dict, peptide_1, peptide_2):

            distance_map = []
            for amino_1 in coordinate_dict[peptide_1]:
                coordinate_1 = coordinate_dict[peptide_1][amino_1]

                distance_map.append([amino_1])

                for amino_2 in coordinate_dict[peptide_2]:
                    coordinate_2 = coordinate_dict[peptide_2][amino_2]

                    euclidean_distance = morphoscanner.distance.get_euclidean_distance(coordinate_1, coordinate_2)
                    distance_map[amino_1].append(euclidean_distance)

                del distance_map[amino_1][0]

            distance_map = np.asarray(distance_map)

            return distance_map

        
        
        # compute distance map and return a n_peptide x n_peptide x n_res x n_res array
        def compute_distance_maps_from_coordinate_dict(coordinate_dict):

            aggregate_distance_map = []

            for peptide_1 in tqdm.tqdm(coordinate_dict):
            #for peptide_1 in coordinate_dict:
                aggregate_distance_map.append([peptide_1])

                #for peptide_2 in tqdm.tqdm(coordinate_dict):
                for peptide_2 in coordinate_dict:
                    distance_map = morphoscanner.distance.compute_distance_map(coordinate_dict, peptide_1, peptide_2)

                    aggregate_distance_map[peptide_1].append(distance_map)

                del aggregate_distance_map[peptide_1][0]

            aggregate_distance_array = np.asarray(aggregate_distance_map)

            return aggregate_distance_array


        # COMPUTE CONTACT MAPS
        # TO DO: parametrize the threshold distance in a better way (e.g. )
        def compute_contact_maps_as_array(distance_maps_array):

            # distance between the first and the second aminoacid of the first chain
            intrapeptide_minimum_distance = distance_maps_array[0][0][0][1] 

            contact_map_list = []

            # contact is in a distance up to 150% of the intrapeptide_minimum_distance [TO IMPROVE!!!]
            threshold_distance = (intrapeptide_minimum_distance * 1.5)

            for model_1 in range(distance_maps_array.shape[0]):
                contact_map_list.append([])
                for model_2 in range(distance_maps_array[model_1].shape[0]):

                    contact_map_list[model_1].append([])

                    if model_1 == model_2:

                        contact_map_list[model_1][model_2].extend(np.zeros((distance_maps_array.shape[2], distance_maps_array.shape[3])))

                    else:

                        contact_map = np.zeros((distance_maps_array.shape[2], distance_maps_array.shape[3]))

                        for chain_1 in range(distance_maps_array[model_1][model_2].shape[0]):

                            for chain_2 in range(distance_maps_array[model_1][model_2][chain_1].shape[0]):

                                distance = distance_maps_array[model_1][model_2][chain_1][chain_2]

                                if distance < threshold_distance:
                                    contact_map[chain_1][chain_2] = 1 #True
                                else:
                                    pass

                        contact_map_list[model_1][model_2].extend(contact_map)

            contact_array = np.asarray(contact_map_list)

            return contact_array
        
        
        
        # get average distance map from distance maps set
        def get_mean_distance_map(distance_maps):
            '''
            Calculate mean distance map from distance maps set

            Argument: distance maps set

            return: np.array with average intrapeptide distance

            '''

            # create array of zeros of shape number_of_residues * number_of_residues
            # depending on peptide residue number ### TO FIX FOR MULTIMONOMERIC ASSEMBLY
            base = np.zeros((distance_maps[0][0].shape[0], distance_maps[0][0].shape[1]))

            # initialize counter
            counter = 0

            # iterate throught peptides in the aggregate
            for peptide_1 in range(distance_maps.shape[0]):
                for peptide_2 in range(distance_maps.shape[1]):

                    # if peptide index are the same (intrapeptide distance map)
                    if peptide_1 == peptide_2:

                        # intrapeptide distance map
                        actual_distance_map = distance_maps[peptide_1][peptide_2]

                        # sum base and current distance map
                        base = base + actual_distance_map

                        #update counter
                        counter += 1

            #for element in base (every element is the sum of distance_map(i,j) for every distance map)
            for row in range(len(base)):
                for col in range(len(base)):

                    # find the mean for every element of the cumulative distance map
                    base[row][col] = (base[row][col])/counter

            return base

        # Singolar Value Decomposition of distance_map
        def decompose_distance_map(distance_map):
            '''Use Singular value decomposition to get

            distance_map.shape[1] dimensional coordinate
            (same n of dimension as the peptide n of residue)

            As described in:
            Mathematical Modeling of Protein Structure Using Distance Geometry
            Jeong-Mi Yoon, Yash Gad, Zhijun Wu

            Argument: distance map (numpy.array 2D)
            return: X : actual decomposition


            '''

            # initialize a zeros matrix of same shape as the input map
            D = np.zeros(distance_map.shape)

            #iterate trought row
            for i in range(distance_map.shape[0]):

                # iterate trought cols
                for j in range(distance_map.shape[1]):

                    # distance between point point i and point j 
                    dij = distance_map[i][j]

                    # distance between point 0 and point j
                    d0j = distance_map[0][j]

                    #distance between point i and point 0
                    di0 = distance_map[i][0]

                    #fill the zeros matrix with the value obtained with this formula
                    D[i][j] = (d0j**2 + di0**2 - dij**2)/2

            # check rank of matrix (should be of rank 3, but it is of rank distance_map.shape[1])
            #rank = np.linalg.matrix_rank(D)

            # Singular value decomposition on the D matrix
            #svd = np.linalg.svd(D)

            svd = np.linalg.svd(D, full_matrices=False)

            # Calculate distance_map.shape[1] dimensional coordinate, but you need 3
            # the non necessary dimension can give data to better reconstruct the peptide structure
            X = svd[0]*np.sqrt(svd[1])


            #return X, svd, D, rank
            return X

        def get_coordinate_from_decomposition(decomposition):
            '''Take decomposition result and convert it into a coordinate vectors dict

            Argument: decomposition results

            return: dict with reconstructed 3d coordinate vector

            '''

            # take only the first three value to compose a 3D coordinate vector
            coordinate = [e[:3] for e in decomposition]

            # initialize empty dict
            reconstructed_coordinate_dict = {}

            # fill the dict with the ccordinate vectors
            for index,coordinate_vector in enumerate(coordinate):
                reconstructed_coordinate_dict[index] = coordinate_vector

            return reconstructed_coordinate_dict


        # reconstruct 3d coordinate from a distance map
        def get_coordinate_from_distance_map(distance_map):
            ''' compute 3d coordinate from distance map

            Argument: distance_map (numpy.array)

            return: dict with 3d coordinate for every alpha-carbon of a peptide

            '''
            # perform singular value decomposition on distance_map (preprocessed)
            decomposed_mean_distance_map = decompose_distance_map(distance_map)


            # get 3D coordinate
            reconstructed_coordinate_dict = get_coordinate_from_decomposition(decomposed_mean_distance_map)

            return reconstructed_coordinate_dict
        
    
    
    
    
    
    
    
    class distance_tensor():
        
        # instantiate 3d tensor with shape n_peptides * n_residues * n_dimension
        def get_coordinate_tensor(coordinate_dict):
    
            #variables wit dict dimension
            dim0 = len(coordinate_dict)
            dim1 = len(coordinate_dict[0])
            dim2 = len(coordinate_dict[0][0])

            #initialize a 0s tensor
            if torch.cuda.is_available() == True:
                
                zero = torch.zeros([dim0,dim1,dim2], dtype=torch.float32, device='cuda')

            else:
                zero = torch.zeros([dim0,dim1,dim2], dtype=torch.float32)


            for peptide in coordinate_dict:

                for aminoacid in coordinate_dict[peptide]:
                    
                    
                    if torch.cuda.is_available() == True:
                        # create torch tensor on cuda device with cordinate [x,y,z...]
                        zero[peptide][aminoacid] = torch.cuda.FloatTensor(coordinate_dict[peptide][aminoacid])

                    else:
                        zero[peptide][aminoacid] = torch.FloatTensor(coordinate_dict[peptide][aminoacid])

                        
                        
            return zero
        
        

        
        #compute euclidean norm, fast
        def compute_euclidean_norm_torch(coordinate_tensor):
            '''Use matrix to compute euclidean distance dataset wise
                and return a set of distance matrix for everi couple of peptides

            ****Runs in parallel on CUDA devices.

            Argument: tensor of shape n_peptide * n_residue * number of dimension (3 for 3d)

            return: tensor of shape n_peptide * n_peptide * n_residue * n_peptide

            '''

            #create tensor of 0s with shape n_pep x n_pep * n_res + n_res
            if torch.cuda.is_available() == True:
                zero = torch.zeros((coordinate_tensor.shape[0], coordinate_tensor.shape[0], coordinate_tensor.shape[1], coordinate_tensor.shape[1]), dtype=torch.float32, device='cuda')
            else:
                zero = torch.zeros((coordinate_tensor.shape[0], coordinate_tensor.shape[0], coordinate_tensor.shape[1], coordinate_tensor.shape[1]), dtype=torch.float32)
            
            #cicle on peptide
            for index1, peptide1 in tqdm.tqdm(enumerate(coordinate_tensor)):

                #cicle on peptide (upper triangle + diagonal)
                for index2 in range(index1, coordinate_tensor.shape[0]):
                #for index2 in range(coordinate_tens.shape[0]):

                    #coordinate vector
                    peptide2 = coordinate_tensor[index2]

                    x_norm = torch.pow(peptide1, 2).sum(1).view(-1,1)
                    y_t = torch.transpose(peptide2, 0, 1)
                    y_norm = torch.pow(peptide2, 2).sum(1).view(1,-1)

                    dist = torch.sqrt(x_norm + y_norm - 2.0 * torch.mm(peptide1, y_t))

                    #dist = x_norm + y_norm - 2.0 * torch.mm(peptide1, y_t)
                    #fine = torch.clamp(dist, 0.0, np.inf) #should be there, but is not working somehow

                    # add distance map in the right position of the 0s tensor
                    zero[index1][index2] = dist

                    # if mesuring between different peptides
                    if index1 != index2:
                        # put transpose of distance map in lower triangle
                        zero[index2][index1] = dist.transpose(1,0)

            #convert nan to 0  (using this instead of torch.clamp())       
            zero[torch.isnan(zero)] = 0
            
            if torch.cuda.is_available():
                # move to system memory and cast to numpy array
                zero = zero.cpu().numpy()
    
            return zero
    
        #https://discuss.pytorch.org/t/efficient-distance-matrix-computation/9065
        #https://www.dropbox.com/h?preview=Parallel+Euclidean+distance+matrix+computation+on+big+datasets.pdf      
        
        
        

        
        
        
        
        
        
        
        
        
    class cross_correlation():
        
        #### ANALYSIS

        def shift_library_maker(contact_map_to_analyze):

            ''' riceve numero di righe e di colonne
            restituisce un array shape((((row + col)*2)-2),row,col).
            ogni slice è una diagonale. Lo stack copre le diagonali su tutta la matrice'''

            row = contact_map_to_analyze.shape[0]
            col = contact_map_to_analyze.shape[1]

            kron_dict = {}
            kron_list_parallel = []
            kron_list_antiparallel = []

            for e in range(-row+1, col):
                array = np.eye(row, col, e)
                kron_list_parallel.append(array)
                kron_list_antiparallel.append(np.fliplr(array))

            kron_array_parallel = np.asarray(kron_list_parallel)
            kron_array_antiparallel = np.asarray(kron_list_antiparallel)

            kron_dict['parallel'] = kron_array_parallel
            kron_dict['antiparallel'] = kron_array_antiparallel

            return kron_dict


        def normalized_cross_correlation_function(contact_map):
            '''
            Calculate normalized cross correlation function between a contact map and an ideal map.

            Arguments : contact map, as output from get_contact_maps function
                        shift_matrix_stack, as output from shift_matrix_maker function

            Return : a list [ncc_value, index (in the shift_matrix_stack) of the shift matrix
                        that is matching the contact map

                    '''
            shift_matrix_library = morphoscanner.cross_correlation.shift_library_maker(contact_map)

            cross_correlation_values = []
            max_val = []
            sum_contact_map = np.sum(contact_map)

            if sum_contact_map < 2:
                pass

            else:
                for sense in shift_matrix_library:
                    for index, z in enumerate(shift_matrix_library[sense]):

                        shift_matrix = shift_matrix_library[sense][index]
                        sum_shift_matrix = np.sum(shift_matrix)
                        ncc_value = (np.sum((contact_map * shift_matrix))/((np.sqrt(sum_contact_map))*(np.sqrt(sum_shift_matrix))))  # normalized cross correlation function of contact matrix and shift matrix
                        cross_correlation_values.append([ncc_value, index, sum_contact_map, sense])

                    max_val = max(cross_correlation_values) # get only the best match (highest value of ncc)

            return max_val



        def normalized_cross_correlation_for_dataset(contact_array):
            '''Calculate normalized cross correlation function between the full contacts map and and the .

            Arguments : contact map, as output from get_contact_maps function
                        shift_matrix_stack, as output from shift_matrix_maker function

            Return : a list [ncc_value, index (in the shift_matrix_stack) of the shift matrix that is matching the contact map'''

            contact_dict = {}

            #for row in tqdm.tqdm(range(contact_array.shape[0])):
            for row in range(contact_array.shape[0]):

                for col in range((row+1), contact_array.shape[1]):
                #for col in range(contact_array.shape[1]):

                    best_match = []
                    best_match = morphoscanner.cross_correlation.normalized_cross_correlation_function(contact_array[row][col])

                    if len(best_match) == 0:
                        pass

                    else:
                        if row in contact_dict:
                            contact_dict[row].append([row, col, best_match])

                        else:
                            contact_dict[row] = [[row, col, best_match]]

            return contact_dict
        
        
    
    
    
    
    

    
    
    class denoise():
        
        
        #denoise dataset. GET SENSE 'PARALLEL' OR 'ANTIPARALLEL'....'NEED TO KNOW AMINOACID OF PEPTIDES'
        def denoise_contact_maps(contact_maps):
            
            '''Denoise the contact_maps dataset using the shift_matrix
            
            Arguments : contact_maps, normalized_cross_correlation_result
            
            return : a dict with key:value = row : row, col, denoised_map
            
            '''
        
            normalized_cross_correlation_results = morphoscanner.cross_correlation.normalized_cross_correlation_for_dataset(contact_maps)
        
        
            denoised_dict = {}
        
            for peptide_1 in normalized_cross_correlation_results:
                denoised_dict[peptide_1] = {}
                for index, peptide_2 in enumerate(normalized_cross_correlation_results[peptide_1]):
        
                    row = peptide_2[0]
                    col = peptide_2[1]
        
        
        
                    contact_map = contact_maps[row][col]
                    sense = peptide_2[2][3]
                    shift_matrix_index = normalized_cross_correlation_results[peptide_1][index][2][1]
        
                    shift_matrix = morphoscanner.cross_correlation.shift_library_maker(contact_map)
                    shift_matrix = shift_matrix[sense][shift_matrix_index]
                    denoised_map = contact_map * shift_matrix
        
                    denoised_dict[row][col] = denoised_map
                    
                    
            full_denoised_dict = {}
            for peptide_1 in tqdm.tqdm(denoised_dict):
                for peptide_2 in denoised_dict[peptide_1]:
                    contact_map = denoised_dict[peptide_1][peptide_2]
        
                    if peptide_1 in full_denoised_dict:
                        full_denoised_dict[peptide_1][peptide_2] = contact_map
        
                    if peptide_1 not in full_denoised_dict:
                        full_denoised_dict[peptide_1] = {peptide_2:contact_map}
        
                    if peptide_2 in full_denoised_dict:
                        full_denoised_dict[peptide_2][peptide_1] = contact_map.T
        
                    if peptide_2 not in full_denoised_dict:
                        full_denoised_dict[peptide_2] = {peptide_1:contact_map.T}
            
            return full_denoised_dict
         
        

        
    
    
    
    
    
    
    class graph():
        
        # graph clustering
        def nx_graph_search(denoised_dict, minimum_contacts = 3):
            
            ''' Graph clustering of peptides in the aggregate.
            
            Input: denoised contact maps dict
            
            return: networkx.MultiGraph
            
            #######
            
            Search for group of minimum 3 peptides (beta_sheet),
            joined at least with 'minimum_contacts'. default = 3
            
            '''
            # Instantiate graph
            graph = nx.MultiGraph()
            
            #iter on peptide
            for peptide_1 in denoised_dict:
                
                #iter on peptide joined to peptide1
                for peptide_2 in denoised_dict[peptide_1]:
                    #retrieve contact_map of peptide1 and peptide2
                    array_1 = denoised_dict[peptide_1][peptide_2]
                    
                    #iter on peptide joined to peptide2
                    for peptide_3 in denoised_dict[peptide_2]:
                        if peptide_3 != peptide_1:
                            #retrieve contact_map of peptide1 and peptide2
                            array_2 = denoised_dict[peptide_2][peptide_3]
                            
                            # create row and column vector from contact maps
                            vect_1 = morphoscanner.math_utility.get_row_vector(array_1)
                            vect_2 = morphoscanner.math_utility.get_col_vector(array_2)
                            
                            #check for contact
                            contacts = np.dot(vect_1, vect_2)
                            
                            #add edge only if there are enough contacts
                            if contacts >= minimum_contacts:

                                graph.add_edge(peptide_1, peptide_2)

                                graph.add_edge(peptide_2, peptide_3)

            return graph

        #A novel graph clustering algorithm based on discrete-time quantum random walk
        #S.G. Roya, A. Chakrabarti

        # working with networkX
 
        # when you add_edge, nodes are created if they are not there
        # you can put info in edge (as distance, n of contacts, contact map)

        
        
        
        # create a full graph
        def full_graph(denoised_dict):
            ''' Create a full graph of all the peptides in the frame.

            Every peptide is a node in the graph.
            Edges join contacting peptides.
            Edges have attribute 'length' that gives you the number of contact between the peptides

            Useful for peptides behaviour analysis during molecular dynamics

            Arguments: denoised contact maps dict
            return: networkx.MultiGraph

            '''
            graph = nx.MultiGraph()

            for peptide_1 in denoised_dict:
                for peptide_2 in denoised_dict[peptide_1]:

                    array_1 = denoised_dict[peptide_1][peptide_2]

                    graph.add_node(peptide_1)
                    graph.add_node(peptide_2)

                    number_of_contacts = array_1.sum()

                    if number_of_contacts >= 1:

                        graph.add_edge(peptide_1, peptide_2, length = number_of_contacts)

            return graph


        #FIND SUBGRAPH
        def find_subgraph(graph):
            '''
            Find subgraph of joined peptides that have no node in common.

            Argument: NetworkX MultiGraph

            Return: list of subgraph ordered from one end to the other

            '''

            subgraph_list = []

            for node in graph:

                # don't explore node that are already in subgraph_list
                if node not in set(nod for nod_list in subgraph_list for nod in nod_list):

                    # tree is the list of nodes joined to node, starting from node
                    # using depht first search
                    tree = [e for e in nx.algorithms.traversal.depth_first_search.dfs_tree(graph, node)]

                    # check if the first node of the tree has adjiacency == 1
                    # so it checks if it is the first or last node of the subgraph
                    if len(graph[tree[0]]) == 1:

                        if len(subgraph_list) == 0:
                            subgraph_list.append(tree)

                        else:
                            # use generator to check if the tree is already in the subgraph
                            if set(tree) not in (set(i) for i in subgraph_list):
                                subgraph_list.append(tree)

            return subgraph_list
        
        
        
        
        
        def get_not_in_subgraph(coordinate_dict, subgraph):
            '''Get peptide alone or in cluster of 2 in a frame.
            
            Input: coordinate_dict, output of 'find_subgraph' function            
            
            Output: list
            
            #####
            
            Basically this function gives you all the node left out
            from the 'find_subgraph' function. That are all the node
            with 0 or 1 neighbour.
            
            '''
            # one line function
            #out = [e for e in coordinate_dict if e not in [a for i in subgraph for a in i]]

            out = []
            
            # get a list with all the node in subgraph
            subgraph = [a for i in subgraph for a in i]

            # iter on all element and get the one that are not in subgraph
            for e in coordinate_dict:

                if e not in subgraph:

                    out.append(e)

            return out
            
    
        
    
    
    
    
        
        
        
              
        
        
        
        
        
    class plot():
        
        
        ########## PLOT PEPTIDE LIST
        # plot a list of peptide point cloud in 3d space.
        # The box axis have arbitrary scale dependent on the aminoacids distance
        # you can select to show the centroid
        def plot_peptide_list(coordinate_dict, peptide_list, centroid=False):
            x = []
            y = []
            z = []
            x_median = float
            y_median = float
            z_median = float


            for peptide in range(len(peptide_list)):
                x.append([peptide])
                y.append([peptide])
                z.append([peptide])
                for aminoacid in coordinate_dict[peptide_list[peptide]]:

                    point = coordinate_dict[peptide_list[peptide]][aminoacid]
                    x[peptide].append(point[0])
                    y[peptide].append(point[1])
                    z[peptide].append(point[2])

                del x[peptide][0]
                del y[peptide][0]
                del z[peptide][0]

            if centroid == True:

                def assemble_coordinate(axis_coordinate_list):
                    median_list = []
                    for coordinate_set in axis_coordinate_list:
                        median = np.median(coordinate_set)
                        median_list.append(median)
                    return median_list

                x_median = assemble_coordinate(x)
                y_median = assemble_coordinate(y)
                z_median = assemble_coordinate(z)


            #%matplotlib notebook

            fig = plt.figure()
            
            ax = plt.axes(projection='3d')
            

            for pep in range(len(x)):

                ax.scatter3D(x[pep],y[pep],z[pep])

                if centroid == True:

                    ax.scatter3D(x_median[pep], y_median[pep], z_median[pep], c='red')


            #return  plt.show(), [x,y,z], [x_median, y_median, z_median]         
            return plt.show()
        
        
        
        #plot from tensor
        def plot_peptide_from_tensor(coordinate_dict, peptide_list, centroid=False):

            x = []
            y = []
            z = []
            x_median = float
            y_median = float
            z_median = float


            for peptide in range(len(peptide_list)):
                x.append([peptide])
                y.append([peptide])
                z.append([peptide])
                for index, aminoacid in enumerate(coordinate_dict[peptide_list[peptide]]):

                    point = coordinate_dict[peptide_list[peptide]][index]
                    x[peptide].append(point[0])
                    y[peptide].append(point[1])
                    z[peptide].append(point[2])

                del x[peptide][0]
                del y[peptide][0]
                del z[peptide][0]

            x = torch.FloatTensor(x)
            y = torch.FloatTensor(y)
            z = torch.FloatTensor(z)

            if centroid == True:

                def assemble_coordinate(axis_coordinate_list):
                    median_list = []
                    for coordinate_set in axis_coordinate_list:
                        median = torch.median(torch.FloatTensor(coordinate_set))
                        print(median)
                        median_list.append(median)
                    return median_list

                x_median = assemble_coordinate(x)
                y_median = assemble_coordinate(y)
                z_median = assemble_coordinate(z)

            #%matplotlib notebook

            fig = plt.figure()

            ax = plt.axes(projection='3d')


            for pep in range(len(x)):

                # scatter points, making list from torch tensor item
                ax.scatter3D([e.item() for e in x[pep]],[e.item() for e in y[pep]],[e.item() for e in z[pep]])

                if centroid == True:

                    ax.scatter3D(x_median[pep].item(), y_median[pep].item(), z_median[pep].item(), c='red')


            #return  plt.show(), [x,y,z], [x_median, y_median, z_median]         
            return plt.show()
        
        
        
        
        # plot single peptide (with autoscaling of axes)
        def plot_single_peptide(peptide_coordinate_dict, centroid=False):
            x = []
            y = []
            z = []

            for residue in peptide_coordinate_dict:
                point = peptide_coordinate_dict[residue]
                x.append(point[0])
                y.append(point[1])
                z.append(point[2])


            x = np.asarray(x)
            y = np.asarray(y)
            z = np.asarray(z)

            fig = plt.figure()
            ax = plt.axes(projection='3d')
            ax.scatter3D(x,y,z, c='b')


            if centroid == True:
                median_centroid = [np.median(x), np.median(y), np.median(z)]
                ax.scatter3D(median_centroid[0], median_centroid[1], median_centroid[2], c='r')

            # Create cubic bounding box to simulate equal aspect ratio
            max_range = np.array([x.max()-x.min(), y.max()-y.min(), z.max()-z.min()]).max()
            Xb = 0.5*max_range*np.mgrid[-1:2:2,-1:2:2,-1:2:2][0].flatten() + 0.5*(x.max()+x.min())
            Yb = 0.5*max_range*np.mgrid[-1:2:2,-1:2:2,-1:2:2][1].flatten() + 0.5*(y.max()+y.min())
            Zb = 0.5*max_range*np.mgrid[-1:2:2,-1:2:2,-1:2:2][2].flatten() + 0.5*(z.max()+z.min())
            # Comment or uncomment following both lines to test the fake bounding box:
            for xb, yb, zb in zip(Xb, Yb, Zb):
                ax.plot([xb], [yb], [zb], 'w')

            return plt.show()
        
    
    # class for high level functionality, premade workflow...
    class high_level():
        
        # automatically infer peptide lenght
        def coordinate_dict_from_gro(gro_file):
            
            cleaned = morphoscanner.read.clean_gro(gro_file)
            
            coordinate = morphoscanner.read.get_coordinate_dict_from_cleaned_gro(cleaned)
            
            return coordinate
        
        
        #specify peptide lenght
        def coordinate_dict_from_gro_fix_len_peptide(gro_file, peptide_lenght):
            
            cleaned = morphoscanner.read.clean_gro(gro_file)
            
            coordinate = morphoscanner.read.get_coordinate_dict_from_cleaned_gro_for_fixed_lenght_peptides(cleaned, peptide_lenght)
            
            return coordinate
        
        
        # compute distance maps, fast with torch.cuda
        def compute_distance_maps_fast_from_gro(gro_file):
            
            coordinate_dict = morphoscanner.high_level.coordinate_dict_from_gro(gro_file)
            
            coordinate_tens = morphoscanner.distance_tensor.get_coordinate_tensor(coordinate_dict)
            
            distance_maps = morphoscanner.distance_tensor.compute_euclidean_norm_torch(coordinate_tens)
            
            return distance_maps
        
        
        # make full graph from distance maps
        def graph_full_from_distance_maps(distance_maps):
            
            contact_maps = morphoscanner.distance.compute_contact_maps_as_array(distance_maps)
            
            denoised = morphoscanner.denoise.denoise_contact_maps(contact_maps)
            
            graph = morphoscanner.graph.full_graph(denoised)
            
            return graph
        
        
            

            
            
            
            
        
        

    


In [ ]:
torch.cuda.is_available()

In [ ]:
# Example with class library
cleaned = morphoscanner.read.clean_gro(trj_gro)
coordinate_dict = morphoscanner.read.get_coordinate_dict_from_cleaned_gro(cleaned)
trj0_coordinate = morphoscanner.read.get_coordinate_dict_from_cleaned_gro_for_fixed_lenght_peptides(cleaned, 12)
#trj0_distance_maps = morphoscanner.distance.compute_distance_maps_from_coordinate_dict(trj0_coordinate)
#trj0_contact_maps = morphoscanner.distance.compute_contact_maps_as_array(trj0_distance_maps)


In [ ]:
trj0_denoised_dict = morphoscanner.denoise.denoise_contact_maps(trj0_contact_maps)
trj0_graph = morphoscanner.graph.nx_graph_search(trj0_denoised_dict)
trj0_subgraph = morphoscanner.graph.find_subgraph(trj0_graph)

In [ ]:
morphoscanner.plot.plot_peptide_list(trj0_coordinate, [e for e in trj0_coordinate])

In [ ]:
# compute distance maps and return a n_peptide x n_peptide x n_res x n_res array ## from .gro coordinate dict
start = timer()
distance_maps_seed_1 = compute_distance_maps_from_coordinate_dict(coordinate_dict_seed_1)
end = timer()
print(end-start)

# compute contact maps from distance maps
start = timer()
contact_maps_seed_1 = compute_contact_maps_as_array(distance_maps_seed_1)
end = timer()
print(end-start)

# compute cross correlation and get values
start = timer()
normalized_cross_correlation_results_seed_1 = normalized_cross_correlation_for_dataset(contact_maps_seed_1)
end = timer()
print(end-start)

# denoise dataset
start = timer()
denoised_dict_seed_1 = denoise_full_dataset(contact_maps_seed_1, normalized_cross_correlation_results_seed_1)
end = timer()
print(end-start)

# recreate full contact map dataset mirroring the upper triangle
# return dict
start = timer()
full_denoised_dict_seed_1 = reconstruct_full_matrix(denoised_dict_seed_1)
end = timer()
print(end-start)

#create graph
start = timer()
graph_seed_1 = nx_graph_search(full_denoised_dict_seed_1)
end = timer()
print(end-start)

#search subgraph
start = timer()
subgrap_list_seed_1 = find_subgraph(graph_seed_1)
end = timer()
print(end-start)

#find mean distance map
start = timer()
mean_distance_map_seed_1 = get_mean_distance_map(distance_maps_seed_1)
end = timer()
print(end-start)

# decompose mean distance matrix
start = timer()
decomposed_mean_distance_map = decompose_distance_map(mean_distance_map_seed_1)
end = timer()
print(end-start)
      
# get coordinate from distance map      
start = timer()
reconstructed_coordinate_dict = get_coordinate_from_decomposition(decomposed_mean_distance_map)
end = timer()
print(end-start)


In [ ]:
trj0_mean_distance = morphoscanner.distance.get_mean_distance_map(trj0_distance_maps)

plt.imshow(trj0_mean_distance)

In [ ]:
bb = u.select_atoms('name BB')
    #trajectory_dict = {}
    #for index, time_steps in enumerate(u.trajectory):
     #   trajectory_dict[index] = bb.positions

In [ ]:
trj0_coordinate[0]

In [ ]:
%matplotlib notebook

In [ ]:
nx.draw_networkx(trj0_graph)

In [ ]:
trj0_subgraph

In [ ]:
# Example with class library

cleaned = morphoscanner.read.clean_gro(trj_gro)
coordinate_dict = morphoscanner.read.get_coordinate_dict_from_cleaned_gro(cleaned)
trj0_coordinate = morphoscanner.read.get_coordinate_dict_from_cleaned_gro_for_fixed_lenght_peptides(cleaned, 12)
trj0_distance_maps = morphoscanner.distance.compute_distance_maps_from_coordinate_dict(trj0_coordinate)
trj0_contact_maps = morphoscanner.distance.compute_contact_maps_as_array(trj0_distance_maps)

In [ ]:
plt.close()

In [ ]:
%matplotlib notebook

In [ ]:
cleaned = morphoscanner.read.clean_gro(trj_gro)
coordinate_dict = morphoscanner.read.get_coordinate_dict_from_cleaned_gro(cleaned)

In [ ]:
coordinate_dict

In [ ]:
def get_coordinate_dict_from_cleaned_gro_for_fixed_lenght_peptides(cleaned_gro_file, peptide_lenght):
    '''Works only with system made of peptide of equal peptide length
    (the peptide in the trajectory have all the same number of aminoacids).
    
    Usefull to disassemble premade aggregate in a system of omogenous peptide lenght'''
    
    peptide_coordinate_dict = {}

    for peptide in range(len(cleaned_gro_file)//peptide_lenght):

        peptide_coordinate_dict[peptide] = {}

        peptide_data = cleaned_gro_file[(peptide*peptide_lenght):((peptide*peptide_lenght)+peptide_lenght)]

        for index, amino in enumerate(peptide_data):

            peptide_coordinate_dict[peptide][index] = [float(amino[3]), float(amino[4]), float(amino[5])] ## can you generalize this 3? Maybe with re or isdigit function??
    
    return peptide_coordinate_dict
        
    
    

        

In [ ]:
trj0_coordinate = morphoscanner.read.get_coordinate_dict_from_cleaned_gro_for_fixed_lenght_peptides(cleaned, 12)

In [ ]:
peptide_lenght=12
for peptide in range(len(cleaned)//peptide_lenght):

        peptide_coordinate_dict[peptide] = {}

        peptide_data = cleaned[(peptide*peptide_lenght):((peptide*peptide_lenght)+peptide_lenght)]

        for index, amino in enumerate(peptide_data):

            peptide_coordinate_dict[peptide][index] = [float(amino[3]), float(amino[4]), float(amino[5])] ## can you generalize this 3? May

In [ ]:
# Example with class library
cleaned = morphoscanner.read.clean_gro(path)
coordinate_dict = morphoscanner.read.get_coordinate_dict_from_cleaned_gro(cleaned)
trj0_coordinate = morphoscanner.read.get_coordinate_dict_from_cleaned_gro_for_fixed_lenght_peptides(cleaned, 12)

start = timer()
trj0_distance_maps = morphoscanner.distance.compute_distance_maps_from_coordinate_dict(trj0_coordinate)
end = timer()
print(end - start)

trj0_contact_maps = morphoscanner.distance.compute_contact_maps_as_array(trj0_distance_maps)


In [ ]:
trj0_denoised_dict = morphoscanner.denoise.denoise_contact_maps(trj0_contact_maps)
trj0_graph = morphoscanner.graph.nx_graph_search(trj0_denoised_dict)
trj0_subgraph = morphoscanner.graph.find_subgraph(trj0_graph)

In [ ]:
trj0_mean_distance = morphoscanner.distance.get_mean_distance_map(trj0_distance_maps)
trj0_decomposition = morphoscanner.distance.decompose_distance_map(trj0_mean_distance)
trj0_mean_coordinate = morphoscanner.distance.get_coordinate_from_decomposition(trj0_decomposition)

In [ ]:
morphoscanner.plot.plot_peptide_list(trj0_coordinate,[0,1,2,3,4,5,6,7])

In [ ]:
%matplotlib notebook

In [ ]:
plt.close()

In [ ]:
#morphoscanner.plot.plot_single_peptide(trj0_mean_coordinate)

In [ ]:
t = morphoscanner.high_level.coordinate_dict_from_gro(path)

In [ ]:
morphoscanner.plot.plot_peptide_list(t, [e for e in t])

In [ ]:
plt.close()

In [ ]:
def get_coordinate_tensor(coordinate_dict):
    
    #variables wit dict dimension
    dim0 = len(coordinate_dict)
    dim1 = len(coordinate_dict[0])
    dim2 = len(coordinate_dict[0][0])

    #initialize a 0s tensor
    zero = torch.zeros([dim0,dim1,dim2], dtype=torch.float32, device='cuda')
    
    
    for peptide in coordinate_dict:
        
        for aminoacid in coordinate_dict[peptide]:
            
            # create toorch tensor on cuda device with cordinate [x,y,z...]
            zero[peptide][aminoacid] = torch.cuda.FloatTensor(coordinate_dict[peptide][aminoacid])
            
    return zero

In [ ]:
coordinate_test = morphoscanner.high_level.coordinate_dict_from_gro()

In [ ]:

def compute_euclidean_norm_torch(coordinate_tensor):
    '''Use matrix to compute euclidean distance dataset wise
        and return a set of distance matrix for everi couple of peptides
        
    ****Runs in parallel on CUDA devices.
    
    Argument: tensor of shape n_peptide * n_residue * number of dimension (3 for 3d)
    
    return: tensor of shape n_peptide * n_peptide * n_residue * n_peptide
    
    '''
    
    #create tensor of 0s with shape n_pep x n_pep * n_res + n_res
    zero = torch.zeros((coordinate_tensor.shape[0], coordinate_tensor.shape[0], coordinate_tensor.shape[1], coordinate_tensor.shape[1]), dtype=torch.float32, device='cuda')
    
    #cicle on peptide
    for index1, peptide1 in tqdm.tqdm(enumerate(coordinate_tens)):
        
        #cicle on peptide (upper triangle + diagonal)
        for index2 in range(index1, coordinate_tens.shape[0]):
        #for index2 in range(coordinate_tens.shape[0]):
            
            #coordinate vector
            peptide2 = coordinate_tens[index2]
            
            x_norm = torch.pow(peptide1, 2).sum(1).view(-1,1)
            y_t = torch.transpose(peptide2, 0, 1)
            y_norm = torch.pow(peptide2, 2).sum(1).view(1,-1)

            dist = torch.sqrt(x_norm + y_norm - 2.0 * torch.mm(peptide1, y_t))
            
            #dist = x_norm + y_norm - 2.0 * torch.mm(peptide1, y_t)
            #fine = torch.clamp(dist, 0.0, np.inf) #should be there, but is not working somehow
            
            # add distance map in the right position of the 0s tensor
            zero[index1][index2] = dist
            
            # if mesuring between different peptides
            if index1 != index2:
                # put transpose of distance map in lower triangle
                zero[index2][index1] = dist.transpose(1,0)
            
    #convert nan to 0  (using this instead of torch.clamp())       
    zero[torch.isnan(zero)] = 0
    
    zero = zero.cpu().numpy()
    
    return zero # .cpu() to move on system memory
#https://discuss.pytorch.org/t/efficient-distance-matrix-computation/9065
#https://www.dropbox.com/h?preview=Parallel+Euclidean+distance+matrix+computation+on+big+datasets.pdf

In [ ]:
coordinate_dict = morphoscanner.high_level.coordinate_dict_from_gro(path)
coordinate_tens = morphoscanner.distance_tensor.get_coordinate_tensor(coordinate_dict)
start = timer()
distance_tens = morphoscanner.distance_tensor.compute_euclidean_norm_torch(coordinate_tens)
end = timer()
print(end-start)


In [ ]:
contact_maps = morphoscanner.distance.compute_contact_maps_as_array(distance_tens)
denoised_map = morphoscanner.denoise.denoise_contact_maps(contact_maps)
graph_full = morphoscanner.graph.full_graph(denoised_map)
#graph = morphoscanner.graph.nx_graph_search(denoised_map)

In [ ]:
subgraph = morphoscanner.graph.find_subgraph(graph)

In [ ]:
subgraph

In [ ]:
out = morphoscanner.graph.get_not_in_subgraph(coordinate_dict, subgraph)

In [ ]:
start = timer()

end = timer()
print(end-start)

In [ ]:
nx.draw_networkx(graph)

In [ ]:
start = timer()
trj0_distance_maps = morphoscanner.distance.compute_distance_maps_from_coordinate_dict(trj0_coordinate)
end = timer()
print(end - start)

In [ ]:
start = timer()
a = compute_euclidean_norm_torch(coordinate_tens)
end = timer()
print(end-start)

In [ ]:
uu = torch.stack((coordinate_tens[0], coordinate_tens[1]),dim=0)

In [ ]:
def get_not_in_subgraph(coordinate_dict, subgraph):
    
    #out = [e for e in coordinate_dict if e not in [a for i in subgraph for a in i]]
    
    out = []
    
    subgraph = [a for i in subgraph for a in i]
    
    for e in coordinate_dict:
        
        if e not in subgraph:
            
            out.append(e)
            
    return out
            
            

#[e for e in coordinate_dict if e not in [a for i in subgraph for a in i]]

In [ ]:
# COMPUTE CONTACT MAPS
# TO DO: parametrize the threshold distance in a better way (e.g. )
def compute_contact_maps_as_array(distance_maps_array):

    # distance between the first and the second aminoacid of the first chain
    intrapeptide_minimum_distance = distance_maps_array[0][0][0][1] 

    contact_map_list = []

    # contact is in a distance up to 150% of the intrapeptide_minimum_distance [TO IMPROVE!!!]
    threshold_distance = (intrapeptide_minimum_distance * 1.5)

    for model_1 in range(distance_maps_array.shape[0]):
        contact_map_list.append([])
        for model_2 in range(distance_maps_array[model_1].shape[0]):

            contact_map_list[model_1].append([])

            if model_1 == model_2:

                contact_map_list[model_1][model_2].extend(np.zeros((distance_maps_array.shape[2], distance_maps_array.shape[3])))

            else:

                contact_map = np.zeros((distance_maps_array.shape[2], distance_maps_array.shape[3]))

                for chain_1 in range(distance_maps_array[model_1][model_2].shape[0]):

                    for chain_2 in range(distance_maps_array[model_1][model_2][chain_1].shape[0]):

                        distance = distance_maps_array[model_1][model_2][chain_1][chain_2]

                        if distance < threshold_distance:
                            contact_map[chain_1][chain_2] = 1 #True
                        else:
                            pass

                contact_map_list[model_1][model_2].extend(contact_map)

    contact_array = np.asarray(contact_map_list)

    return contact_array

In [ ]:
denoised_map

In [ ]:
# create a full graph
def full_graph(denoised_dict):
    ''' Create a full graph of all the peptides in the frame.
    
    Every peptide is a node in the graph.
    Edges join contacting peptides.
    Edges have attribute 'length' that gives you the number of contact between the peptides
    
    Useful for peptides behaviour analysis during molecular dynamics
    
    Arguments: denoised contact maps dict
    return: networkx.MultiGraph
    
    '''
    graph = nx.MultiGraph()

    for peptide_1 in denoised_dict:
        for peptide_2 in denoised_dict[peptide_1]:
            
            array_1 = denoised_dict[peptide_1][peptide_2]

            graph.add_node(peptide_1)
            graph.add_node(peptide_2)

            number_of_contacts = array_1.sum()

            if number_of_contacts >= 1:

                graph.add_edge(peptide_1, peptide_2, length = number_of_contacts)
    
    return graph

In [ ]:
nx.draw_networkx(graph)

In [ ]:
plt.close()

In [ ]:
%matplotlib notebook

In [ ]:
coordinate_dict = morphoscanner.high_level.coordinate_dict_from_gro_fix_len_peptide(trj_gro,12)
coordinate_tens = morphoscanner.distance_tensor.get_coordinate_tensor(coordinate_dict)
start = timer()
distance_tens = morphoscanner.distance_tensor.compute_euclidean_norm_torch(coordinate_tens)
end = timer()
print(end-start)


In [ ]:
coordinate_dict = morphoscanner.high_level.coordinate_dict_from_gro(trj_gro)

In [ ]:
### TRAJECTORY

# using mdanalysis module


#create Universe from a .gro with coordinates and an .xtc with the trajectory data
u = mda.Universe(trj_gro,trj_xtc)

In [ ]:
# create a dict from a Universe in which each entry is a timestep of the MD simulation
def create_trajectory_dict(universe):
    bb = universe.select_atoms('name BB')
    trajectory_dict = {}
    for index, time_steps in enumerate(universe.trajectory):
        trajectory_dict[index] = bb.positions
    return trajectory_dict

# make trajectory dict
trajectory_dict = create_trajectory_dict(u)

In [ ]:
len(trajectory_dict[0])

In [ ]:
def make_universe(trj_gro, trj_xtc):
    ''' Leverage MDAnalysis.Universe() to parse trajectory file from gromacs output.
    
    Intput: string: system path of gro file (topology) and
                    system path of xtc file (trajectory)
                    of the file to analyze
                    
    return: MDAnalysis.Universe()'''
    
    
    universe = mda.Universe(trj_gro,trj_xtc)
    
    return universe

In [ ]:
morphoscanner.plot.plot_peptide_list(coordinate_dict[0])

In [ ]:
morphoscanner.plot.plot_peptide_list(coordinate_dict, [e for e in coordinate_dict])

In [ ]:
def get_coordinate_from_gro_all_atom(topology, universe):
    
    
    peptides_list = morphoscanner.topology.get_peptide_lenght_list(topology)

    trj_dict = {}

    for peptide, n_res in enumerate(peptides_list[:]):

        trj_dict[peptide] = {}

        if peptide != 0:

            counter += peptides_list[peptide-1]

        else:
            counter = 0


        for res in range(n_res):

            res_name = (str(universe.residues[res + counter]).split()[1].split(',')[0])#.split(',')[0])
            res_position = (str(universe.residues[res + counter]).split()[2].split('>')[0])#.split(',')[0])
            res_id = res_position + '_' + res_name
            
            trj_dict[peptide][res_id] = {}

            for index, atom in enumerate(universe.residues[res + counter].atoms):

                atom_type = str(atom).split()[2]

                atom_number = int(str(atom).split()[1].split(':')[0])

                coordinate = universe.atoms[atom_number].position

                trj_dict[peptide][res_id][atom_type] = coordinate
                
    return trj_dict


    
  

In [ ]:
def get_coordinate_dict_for_fixed_lenght_topology(topology, universe, n_residues = int):

    peptides_list = morphoscanner.topology.get_peptide_lenght_list(trj_gro)

    #n_residues = 12

    n_pep = sum([(e//n_residues) for e in peptides_list])

    trj_dict = {}

    for peptide in range(n_pep):

        trj_dict[peptide] = {}

        if peptide != 0:

            counter += n_residues

        else:
            counter = 0


        for res in range(n_residues):

            res_name = (str(universe.residues[res + counter]).split()[1].split(',')[0])#.split(',')[0])
            res_position = (str(universe.residues[res + counter]).split()[2].split('>')[0])#.split(',')[0])
            res_id = res_position + '_' + res_name


            for index, atom in enumerate(universe.residues[res + (counter)].atoms):

                atom_number = (int(str(atom).split()[1].split(':')[0]) - 1)

                coordinate = universe.atoms[atom_number].position
                position = len(trj_dict[peptide])
                
                trj_dict[peptide][position] = coordinate
                
    return trj_dict

In [ ]:
coordinate_dict

In [ ]:
trj_dict = get_coordinate_dict_for_fixed_lenght_topology(trj_gro, u, 12)

In [ ]:
trj_dict = get_coordinate_from_gro_all_atom(trj_gro, u)

In [ ]:
def get_peptide_lenght_list(topology):
    
    peptide_lenght_list = []

    temporary_list = []

    # iterate trough topology
    for residue in topology:

        # if temporary list just started, add aminoacid position in chain
        if len(temporary_list) == 0:
            temporary_list.append(int(residue[1]))

        else:
            # if position of actual residue is less than last residue
            if temporary_list[-1] > int(residue[1]):

                # append lenght of last peptide to peptide lenght list
                peptide_lenght_list.append(len(temporary_list))

                # empty temporary list
                temporary_list = []

                # append actual residue position
                temporary_list.append(int(residue[1]))

            # if position of actual residue is higher than last residue, ad current residue position
            else:
                temporary_list.append(int(residue[1]))

    # append last peptide lenght to lenght stack
    peptide_lenght_list.append(len(temporary_list))

    return peptide_lenght_list

In [ ]:

u = morphoscanner.topology.make_universe(trj_gro, trj_xtc)

In [ ]:
morphoscanner.plot.plot_peptide_list(trj_dict, [0,1,2,3,4,5,6,7,8,9,10,11,12])

In [ ]:
b = morphoscanner.read.get_peptide_lenght_list(trj_gro)

In [ ]:
%matplotlib notebook

In [ ]:
trj_dict = get_coordinate_dict_for_fixed_lenght_topology(trj_gro, u, 12)

In [ ]:
coord_tens = morphoscanner.distance_tensor.get_coordinate_tensor(trj_dict)

In [ ]:
len(trj_dict)

In [ ]:
coord_tens.shape

In [ ]:
start = timer()
distances = morphoscanner.distance_tensor.compute_euclidean_norm_torch(coord_tens)
end = timer()
print(end - start)

In [ ]:
distances.shape

In [ ]:
plt.imshow(distances[0][0])

In [ ]:
plt.close()

In [ ]:
u.atoms[0]

In [ ]:
cleaned = morphoscanner.topology.get_peptide_lenght_list(trj_gro)

In [ ]:
cleaned

In [6]:
#def get_coordinate_from_gro_all_atom(topology, universe):
    
topology = trj_gro

universe = morphoscanner.topology.make_universe(trj_gro, trj_xtc)

peptides_list = morphoscanner.topology.get_peptide_lenght_list(topology)

trj_dict = {}

for peptide, n_res in enumerate(peptides_list[:]):

    trj_dict[peptide] = {}

    if peptide != 0:

        counter += peptides_list[peptide-1]

    else:
        counter = 0


    for res in range(n_res):

        res_name = (str(universe.residues[res + counter]).split()[1].split(',')[0])#.split(',')[0])
        res_position = (str(universe.residues[res + counter]).split()[2].split('>')[0])#.split(',')[0])
        res_id = res_position + '_' + res_name

        trj_dict[peptide][res_id] = {}

        for index, atom in enumerate(universe.residues[res + counter].atoms):

            atom_type = str(atom).split()[2]

            atom_number = int(str(atom).split()[1].split(':')[0])

            coordinate = universe.atoms[atom_number].position

            trj_dict[peptide][res_id][atom_type] = coordinate
                
    #return trj_dict

192300it [00:00, 2613777.41it/s]
100%|██████████| 192297/192297 [00:00<00:00, 1728132.08it/s]


30

In [ ]:
print('__Number CUDA Devices:', torch.cuda.device_count())

In [10]:
#def get_coordinate_dict_for_fixed_lenght_topology(topology, universe, n_residues = int):
universe = morphoscanner.topology.make_universe(trj_gro, trj_xtc)

topology = trj_gro

def dataframe_from_topology(topology, universe, n_residues):
    
    
    peptides_list = morphoscanner.topology.get_peptide_lenght_list(trj_gro)

    n_residues = 12

    n_pep = sum([(e//n_residues) for e in peptides_list])

    #columns_name = ['atom_number','peptide_number', 'residue_name', 'residue_position', 'coordinates']
    columns_name = ['peptide_number', 'residue_position', 'residue_name', 'atom_position', 'atom_type', 'coordinates']

    # create list for a pd.DataFrame
    # as suggested in https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.append.html
    for_pandas = []

    trj_dict = {}
    
    

    for peptide in range(n_pep):

        trj_dict[peptide] = {}

        if peptide != 0:

            counter += n_residues

        else:
            counter = 0


        for res in range(n_residues):

            res_name = (str(universe.residues[res + counter]).split()[1].split(',')[0])#.split(',')[0])
            res_position = int(str(universe.residues[res + counter]).split()[2].split('>')[0])#.split(',')[0])
            res_id = str(res_position) + '_' + res_name

            #print(str(universe.residues[res + counter]))


            for index, atom in enumerate(universe.residues[res + (counter)].atoms):

                #print(atom)

                atom_number = (int(str(atom).split()[1].split(':')[0]) - 1)

                atom_type = str(atom).split()[2]

                #print(atom_number)
                #print(atom_type)

                #print(str(atom).split())

                coordinate = universe.atoms[atom_number].position
                position = len(trj_dict[peptide])

                trj_dict[peptide][position] = coordinate

                #features = [atom_number,peptide, res_name, position, coordinate]
                features = [peptide, res_position, res_name, position, atom_type, coordinate]

                for_pandas.append(features)



    df = pd.DataFrame(for_pandas,columns=columns_name)
    
    return df
            
            

#return trj_dict

In [11]:
df = dataframe_from_topology(trj_gro, universe, 12)

192300it [00:00, 2537468.02it/s]
100%|██████████| 192297/192297 [00:00<00:00, 1544568.56it/s]


In [12]:
df

,peptide_number,residue_position,residue_name,atom_position,atom_type,coordinates
0,0,1,LEU,0,BB,"[168.961, 85.78, 219.44]"
1,0,1,LEU,1,SC1,"[171.48898, 86.259995, 220.07]"
2,0,2,ASP,2,BB,"[166.68298, 87.723, 217.221]"
3,0,2,ASP,3,SC1,"[164.937, 86.036995, 216.439]"
4,0,3,LEU,4,BB,"[168.461, 90.76, 215.9]"
...,...,...,...,...,...,...
2695,99,11,LEU,22,BB,"[87.799995, 90.85999, 170.07999]"
2696,99,11,LEU,23,SC1,"[88.96999, 87.76, 170.13]"
2697,99,12,LYS,24,BB,"[90.579994, 92.49, 171.01]"
2698,99,12,LYS,25,SC1,"[91.479996, 95.17, 169.4]"


In [ ]:
#coordinate_dict = morphoscanner.high_level.coordinate_dict_from_gro(trj_gro)
coordinate_dict = morphoscanner.topology.
coordinate_tens = morphoscanner.distance_tensor.get_coordinate_tensor(coordinate_dict)
start = timer()
distance_tens = morphoscanner.distance_tensor.compute_euclidean_norm_torch(coordinate_tens)
end = timer()
print(end-start)

In [13]:
trajectory_dict = {}
for index, ts in enumerate(universe.trajectory):
    print("Frame: {0:5d}, Time: {1:8.3f} ps".format(ts.frame, universe.trajectory.time))
    print("Rgyr: {0:g} A".format(universe.atoms.radius_of_gyration()))
    print(universe.atoms.positions)
    positions = universe.atoms.positions
    trajectory_dict[index] = positions

Frame:     0, Time:    0.000 ps
Rgyr: 141.565 A
[[168.961     85.78     219.44    ]
 [171.48898   86.259995 220.07    ]
 [166.68298   87.723    217.221   ]
 ...
 [271.24     206.26      71.7     ]
 [275.18     179.23001   50.6     ]
 [257.97     204.81     104.02    ]]
Frame:     1, Time: 10000.000 ps
Rgyr: 141.885 A
[[172.58699  107.994995 211.28198 ]
 [174.33199  106.065994 210.774   ]
 [173.571    111.474    210.45999 ]
 ...
 [ 15.231999 277.10498   81.862   ]
 [180.76999   81.706    136.55399 ]
 [274.09     183.48     112.536   ]]
Frame:     2, Time: 20000.000 ps
Rgyr: 141.877 A
[[187.256    116.248    226.07199 ]
 [188.81198  114.105995 226.17099 ]
 [185.662    119.46999  227.034   ]
 ...
 [  3.162      5.621     92.95    ]
 [229.03299  127.173996  76.849   ]
 [231.25598  158.105    118.819   ]]
Frame:     3, Time: 30000.000 ps
Rgyr: 141.883 A
[[203.521   126.696   233.87299]
 [204.78899 124.89099 232.40298]
 [203.09799 130.071   235.26   ]
 ...
 [ 91.873   263.97998  69.904  ]
 [

Rgyr: 141.898 A
[[172.26099  117.489     36.632   ]
 [174.363    118.729004  35.6     ]
 [170.392    117.463     33.395   ]
 ...
 [208.54999  202.19     122.298996]
 [258.969    150.489     31.603   ]
 [275.93698  133.11     232.06699 ]]
Frame:    33, Time: 330000.000 ps
Rgyr: 141.9 A
[[169.51999  113.7       37.644   ]
 [171.53     114.224     35.997997]
 [166.80199  114.711     35.369   ]
 ...
 [115.37499  177.288    136.13    ]
 [ 14.551    114.889     93.916   ]
 [210.07399  142.754    259.52298 ]]
Frame:    34, Time: 340000.000 ps
Rgyr: 141.886 A
[[166.015     120.856      46.566998 ]
 [168.006     122.482994   47.209    ]
 [163.76      121.444      43.677002 ]
 ...
 [141.75299   164.69699   161.85199  ]
 [  5.8029995  72.903      95.223    ]
 [131.712     103.96999   278.702    ]]
Frame:    35, Time: 350000.000 ps
Rgyr: 141.841 A
[[159.57399  108.548996  17.001999]
 [161.109    110.396996  15.882999]
 [156.134    109.84699   16.123999]
 ...
 [131.351     96.818    272.33    ]
 [ 

Rgyr: 141.81 A
[[105.378    125.313995 262.77698 ]
 [105.239    127.371994 264.44    ]
 [102.918    122.438995 262.922   ]
 ...
 [105.751     67.862    148.383   ]
 [239.212    130.231      9.889999]
 [263.634    235.91998  195.81299 ]]
Frame:    67, Time: 670000.000 ps
Rgyr: 141.848 A
[[109.897995 111.326    264.546   ]
 [110.256996 113.694    265.68    ]
 [106.314    110.358    264.48798 ]
 ...
 [ 43.487    102.597    141.864   ]
 [231.094    160.709    127.864   ]
 [205.306    243.631    116.20999 ]]
Frame:    68, Time: 680000.000 ps
Rgyr: 141.849 A
[[110.383995 107.496994 -16.567   ]
 [112.336    108.508995 -15.088999]
 [108.991    104.379    -15.095999]
 ...
 [ 68.458    103.40799  216.40999 ]
 [195.27701  214.261    193.12198 ]
 [259.263    267.974    151.151   ]]
Frame:    69, Time: 690000.000 ps
Rgyr: 141.844 A
[[ 93.813    124.506    -11.785   ]
 [ 95.33     125.246994  -9.743   ]
 [ 91.925    121.41999  -11.171   ]
 ...
 [ 51.271996  91.437996 239.85399 ]
 [137.785    240.474

Rgyr: 141.67 A
[[ 65.471    133.819     53.631996]
 [ 66.160995 132.986     51.212997]
 [ 67.647995 132.484     56.334995]
 ...
 [181.826     92.895996 191.94    ]
 [242.87     273.09      16.446999]
 [262.934    189.048    111.187   ]]
Frame:    99, Time: 990000.000 ps
Rgyr: 141.731 A
[[ 77.239    136.071     59.572998]
 [ 77.731    135.736     56.991   ]
 [ 78.953995 134.129     62.322998]
 ...
 [164.056     51.758    227.02599 ]
 [238.31601  249.487     95.723   ]
 [275.228    110.465996  54.78    ]]
Frame:   100, Time: 1000000.000 ps
Rgyr: 141.694 A
[[ 76.196    129.181     47.866   ]
 [ 75.389    127.796005  45.755997]
 [ 79.644    129.999     49.051   ]
 ...
 [ 89.768     15.797     27.925   ]
 [188.491    228.40898   98.702995]
 [271.21     129.948     34.674   ]]
Frame:   101, Time: 1010000.000 ps
Rgyr: 141.688 A
[[ 79.657    132.28      44.392   ]
 [ 77.992    132.941     42.439   ]
 [ 82.337    130.07399   45.566998]
 ...
 [ 93.634995  32.193     65.761   ]
 [194.41399  251.7

Frame:   130, Time: 1300000.000 ps
Rgyr: 141.749 A
[[110.01199  102.513     71.275   ]
 [110.551994 101.369     68.947   ]
 [112.019    101.144     74.12299 ]
 ...
 [149.23099  272.53      17.428   ]
 [280.71698   46.309     45.716   ]
 [ 45.45      70.603    259.74698 ]]
Frame:   131, Time: 1310000.000 ps
Rgyr: 141.756 A
[[107.91599  110.17799   66.598   ]
 [108.727    109.283     64.24    ]
 [108.77     108.912     69.905   ]
 ...
 [ 51.714    257.53598   60.482998]
 [265.193    263.95898   76.476   ]
 [ 50.508     30.530998   9.399   ]]
Frame:   132, Time: 1320000.000 ps
Rgyr: 141.794 A
[[ 95.618    109.60999   69.336   ]
 [ 96.762    109.06999   67.006996]
 [ 98.791    108.895     71.154   ]
 ...
 [ 38.514    243.657    128.534   ]
 [ 46.998997 208.82399  196.334   ]
 [ 14.951    245.63098  265.473   ]]
Frame:   133, Time: 1330000.000 ps
Rgyr: 141.82 A
[[ 90.591995  99.909     93.569   ]
 [ 91.982    100.489     91.389   ]
 [ 93.423     99.756996  95.991   ]
 ...
 [130.422    245.9

In [20]:
trajectory_dict[0]

array([ 38.09, 261.86,  53.85], dtype=float32)

In [26]:
#def get_coordinate_dict_for_fixed_lenght_topology(topology, universe, n_residues = int):
def get_dataframe_from_trajectory(trj_gro, trj_xtc):
    
    universe = morphoscanner.topology.make_universe(trj_gro, trj_xtc)

    topology = trj_gro

    #def dataframe_from_topology(topology, universe, n_residues = int):


    peptides_list = morphoscanner.topology.get_peptide_lenght_list(trj_gro)

    n_residues = 12 ### add autosearch if not specified
    n_pep = sum([(e//n_residues) for e in peptides_list])

    #columns_name = ['atom_number','peptide_number', 'residue_name', 'residue_position', 'coordinates']
    columns_name = ['time_step','peptide_number', 'residue_position', 'residue_name', 'atom_position', 'atom_type', 'coordinates']

    # create list for a pd.DataFrame
    # as suggested in https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.append.html
    for_pandas = []

    trj_dict = {}

    for index_ts, ts in tqdm.tqdm(enumerate(universe.trajectory)):

        trj_dict[index_ts] = {}

        for peptide in range(n_pep):

            trj_dict[index_ts][peptide] = {}

            if peptide != 0:

                counter += n_residues

            else:
                counter = 0


            for res in range(n_residues):

                res_name = (str(universe.residues[res + counter]).split()[1].split(',')[0])#.split(',')[0])
                res_position = int(str(universe.residues[res + counter]).split()[2].split('>')[0])#.split(',')[0])
                res_id = str(res_position) + '_' + res_name

                #print(str(universe.residues[res + counter]))


                for index, atom in enumerate(universe.residues[res + (counter)].atoms):

                    #print(atom)

                    atom_number = (int(str(atom).split()[1].split(':')[0]) - 1)

                    atom_type = str(atom).split()[2]

                    #print(atom_number)
                    #print(atom_type)

                    #print(str(atom).split())

                    coordinate = universe.atoms[atom_number].position
                    position = len(trj_dict[index_ts][peptide])

                    trj_dict[index_ts][peptide][position] = coordinate

                    #features = [atom_number,peptide, res_name, position, coordinate]
                    features = [index_ts, peptide, res_position, res_name, position, atom_type, coordinate]

                    for_pandas.append(features)


    #start = timer()
    df = pd.DataFrame(for_pandas,columns=columns_name)
    #end = timer()
    #print(end-start)
    return df
            
            

#return trj_dict

In [ ]:
a = df.groupby('time_step')

In [ ]:
distribute python libraryfor i in 

In [ ]:
len(trj_dict)

In [ ]:
 for i in a.coordinates:
        i[1]

In [ ]:
df

In [ ]:
a

In [25]:
morphoscanner.plot.plot_peptide_from_tensor(trajectory_dict[0], [122])

IndexError: invalid index to scalar variable.

In [ ]:
a.get_group(2)

In [27]:
df = get_dataframe_from_trajectory(trj_gro, trj_xtc)

192300it [00:00, 2236047.40it/s]
100%|██████████| 192297/192297 [00:00<00:00, 1675210.92it/s]
151it [00:31,  4.79it/s]


In [28]:
df

,time_step,peptide_number,residue_position,residue_name,atom_position,atom_type,coordinates
0,0,0,1,LEU,0,BB,"[168.961, 85.78, 219.44]"
1,0,0,1,LEU,1,SC1,"[171.48898, 86.259995, 220.07]"
2,0,0,2,ASP,2,BB,"[166.68298, 87.723, 217.221]"
3,0,0,2,ASP,3,SC1,"[164.937, 86.036995, 216.439]"
4,0,0,3,LEU,4,BB,"[168.461, 90.76, 215.9]"
...,...,...,...,...,...,...,...
407695,150,99,11,LEU,22,BB,"[36.459, 230.317, 170.70999]"
407696,150,99,11,LEU,23,SC1,"[37.32, 229.527, 173.814]"
407697,150,99,12,LYS,24,BB,"[34.951, 232.99799, 168.845]"
407698,150,99,12,LYS,25,SC1,"[35.388, 232.115, 165.879]"


In [ ]:
plt.close()

In [ ]:
distance_maps = morphoscanner.high_level.compute_distance_maps_fast_from_gro(trj_gro)

In [ ]:
plt.imshow(distance_maps[0][0])

In [ ]:
grafo = morphoscanner.high_level.graph_full_from_distance_maps(distance_maps)

In [ ]:
distance_maps = morphoscanner.read.

In [29]:
cleaned = morphoscanner.read.clean_gro(trj_gro)

192300it [00:00, 2054638.19it/s]
100%|██████████| 192297/192297 [00:00<00:00, 1661360.02it/s]


In [30]:
cleaned

[['1', '1', 'LEU', '16.896', '8.578', '21.944'],
 ['3', '2', 'ASP', '16.668', '8.772', '21.722'],
 ['5', '3', 'LEU', '16.846', '9.076', '21.590'],
 ['7', '4', 'LYS', '16.739', '9.427', '21.557'],
 ['10', '5', 'LEU', '16.797', '9.729', '21.764'],
 ['12', '6', 'ASP', '16.717', '10.077', '21.716'],
 ['14', '7', 'LEU', '16.971', '10.348', '21.679'],
 ['16', '8', 'LYS', '16.860', '10.698', '21.616'],
 ['19', '9', 'LEU', '17.119', '10.953', '21.655'],
 ['21', '10', 'ASP', '16.938', '11.276', '21.595'],
 ['23', '11', 'LEU', '17.159', '11.575', '21.512'],
 ['25', '12', 'LYS', '17.187', '11.945', '21.456'],
 ['28', '13', 'LEU', '17.715', '8.314', '21.512'],
 ['30', '14', 'ASP', '17.658', '8.382', '21.883'],
 ['32', '15', 'LEU', '17.752', '8.736', '21.879'],
 ['34', '16', 'LYS', '17.945', '8.990', '22.068'],
 ['37', '17', 'LEU', '17.872', '9.354', '22.042'],
 ['39', '18', 'ASP', '18.069', '9.646', '22.155'],
 ['41', '19', 'LEU', '17.858', '9.956', '22.181'],
 ['43', '20', 'LYS', '18.096', '10.22